In [1]:
from lib import *

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp

In [1465]:
T       = 60-38+1
n_draw  = 1000
ρ       = .03
σ       = .1
nburn   = 20
p       = .5
yh      = (1+ρ+.1*np.sqrt(2/np.pi))/10
yl      = (1+ρ-.1*np.sqrt(2/np.pi))/10

γ      = 3
δ      = .95
β      = .8
α      = 1.5
Rs     = 1+.022
Rb     = 1+.197

In [1466]:
num_results = T+nburn
dim = n_draw
sample = tfp.mcmc.sample_halton_sequence(
  dim,
  num_results=num_results,
  seed=575)
y_eps = norm.ppf(sample)*σ


In [1467]:

y_pro = np.zeros(y_eps.shape)
y_pro[0, :]  = 1+y_eps[0, :]


for i in range(1, T+nburn):
    y_pro[i, :] = y_pro[i-1, :]*(1+ρ)+y_eps[i, :]
    
y_pro = y_pro[nburn:, :]

In [1468]:
# ave income at age 38: 1.8
# mean(range) log(1+y/s) from data: .25 (.05, .6)  

y_s = np.linspace(.8, 2.8, 50)
s_grid = 1.8/y_s
s_grid.sort()

# mean(range) log(1+b/y) from data: .25 (.05, .6)  

b_y = np.linspace(-4.6, -0.6, 49)
b_grid = np.exp(b_y)*1.8
b_grid.sort()

b_grid = np.insert(b_grid, 0, 0)

In [1469]:
y = 1.03

In [1470]:
N_grid_a   = 50
N_grid_b   = 50
N_grid_tot = N_grid_a*N_grid_b


a_grid0 = np.linspace(1, 10, N_grid_a).reshape(N_grid_a,1)
b_grid0 = np.linspace(0, 10, N_grid_b).reshape(N_grid_b,1)

# change N_grid_a X N_grid_b grid to N_grid_a*N_grid_b X 1 
# value matrix has (a_{t,1}, b_{t-1,1}), (a_{t,2}, b_{t-1,2}) .....

a_grid = np.repeat(a_grid0, N_grid_b, axis=0)
b_grid = np.tile(b_grid0, (N_grid_a,1))

In [1471]:
ct_mat       = np.zeros((N_grid_tot, N_grid_tot))
ct_mat2h     = np.zeros((N_grid_tot, N_grid_tot))
ct_mat2l     = np.zeros((N_grid_tot, N_grid_tot))

ct_mat0h     = np.zeros((N_grid_tot, N_grid_tot))
ct_mat0l     = np.zeros((N_grid_tot, N_grid_tot))


V_tuple      = np.zeros((N_grid_tot, N_grid_tot))
pol_func_a   = np.zeros((N_grid_tot, T+1))
pol_func_b   = np.zeros((N_grid_tot, T+1))
pol_func_c   = np.zeros((N_grid_tot, T+1))


VT           = np.tile(a_grid.T,(N_grid_tot, 1))
Vt           = np.tile(a_grid,(1, N_grid_tot))
#Vt           = np.tile(a_grid.T,(N_grid_tot,1))



y = 1.03

In [1472]:
def u(x) :
    return ((x)**(1-γ)-10)/(1-γ)

In [1473]:
# c = a-(a' + Rb b)/Rs 
for i in range(N_grid_tot):
    ct_mat[:,i] = (a_grid-(a_grid[i]+Rb*b_grid)/Rs).T
ct_mat[ct_mat<0] = np.nan

# c = a-(a'-y'-b' + Rb b)/Rs 

for i in range(N_grid_tot):
    ct_mat2h[:,i] = (a_grid-(a_grid[i]+Rb*b_grid-yh/10-b_grid[i])/Rs).T
    ct_mat2l[:,i] = (a_grid-(a_grid[i]+Rb*b_grid-yl/10-b_grid[i])/Rs).T
    
    ct_mat0h[:,i] = (a_grid-(a_grid[i]-yh/10-b_grid[i])/Rs).T
    ct_mat0l[:,i] = (a_grid-(a_grid[i]-yl/10-b_grid[i])/Rs).T
    
ct_mat2h[ct_mat2h<0] = np.nan
ct_mat2l[ct_mat2l<0] = np.nan
ct_mat0h[ct_mat0h<0] = np.nan
ct_mat0l[ct_mat0l<0] = np.nan



In [1474]:
aT = np.tile(a_grid.T,(N_grid_tot,1))
aT[aT<0] = np.nan
vmax      = 20*u(aT/20)
vmax1     = 20*u(aT/20)


pol_func_a[:,T-1] = a_grid.T
pol_func_b[:,T-1] = b_grid.T

V_tuple    = u(ct_mat) + u(α*Vt) + β*δ*vmax1
V_tuple1   = u(ct_mat) + u(α*Vt) + δ*vmax1
a = vmax1.copy()

vmax    = np.nanmax(V_tuple,1)
vmax1   = vmax.copy()

for i in range(N_grid_tot):
    pol_func_a[i,T] = np.nan
    pol_func_b[i,T] = np.nan
    pol_func_c[i,T-1] = np.nan
    vmax1[i]        = np.nan     
    
    
    if ~np.isnan(vmax[i]):
        pol_func_a[i,T] = a_grid[np.where(vmax[i]== V_tuple[i,:])[0][0]]
        pol_func_b[i,T] = b_grid[np.where(vmax[i]== V_tuple[i,:])[0][0]]
        pol_func_c[i,T-1] = ct_mat[i,np.where(vmax[i]== V_tuple[i,:])[0][0]]
        vmax1[i]        = V_tuple1[i,np.where(vmax[i]== V_tuple[i,:])[0][0]]





In [1475]:
for t in range(22, 0, -1):
    pol_func_a0 = pol_func_a.copy()
    pol_func_b0 = pol_func_b.copy()
    vmax_grid   = np.tile(vmax1,(N_grid_tot,1))
    V_tuple     = .5*(u(ct_mat2h)+u(ct_mat2l)) + u(α*Vt) + β*δ*vmax_grid
    V_tuple1    = .5*(u(ct_mat2h)+u(ct_mat2l)) + u(α*Vt) + δ*vmax_grid
    
    
    vmax    = np.nanmax(V_tuple,1)
    vmax1   = vmax.copy()
    
    pol_func_a[:,t-1] = a_grid.T
    pol_func_b[:,t-1] = b_grid.T

    for i in range(N_grid_tot):
        pol_func_a[i,t] = np.nan
        pol_func_b[i,t] = np.nan
        pol_func_c[i,t-1] = np.nan
        vmax1[i]        = np.nan     
        
        if ~np.isnan(vmax[i]):
            choice = np.where(vmax[i] == V_tuple[i,:])[0][-1]
            pol_func_a[i,t:]  = pol_func_a0[choice,t:]
            pol_func_b[i,t:]  = pol_func_b0[choice,t:]
            pol_func_c[i,t-1] = (ct_mat2h[i,choice]+ct_mat2l[i,choice])/2
            vmax1[i]          = V_tuple1[i,np.where(vmax[i] == V_tuple[i,:])[0][-1]]

    

In [1476]:
pol_func_a0 = pol_func_a.copy()
pol_func_b0 = pol_func_b.copy()
vmax_grid   = np.tile(vmax1,(N_grid_tot,1))
V_tuple     = .5*(u(ct_mat0h)+u(ct_mat0l)) + u(α*Vt) + β*δ*vmax1
V_tuple1    = .5*(u(ct_mat0h)+u(ct_mat0l)) + u(α*Vt) + δ*vmax1


vmax    = np.nanmax(V_tuple,1)
vmax1   = vmax.copy()

pol_func_a[:,0] = a_grid.T
pol_func_b[:,0] = 0

for i in range(N_grid_tot):
    pol_func_a[i,1] = np.nan
    pol_func_b[i,1] = np.nan
    vmax1[i]        = np.nan     

    if ~np.isnan(vmax[i]):
        pol_func_a[i,1:] = pol_func_a0[np.where(vmax[i] == V_tuple[i,:])[0][0],1:]
        pol_func_b[i,1:] = pol_func_b0[np.where(vmax[i] == V_tuple[i,:])[0][0],1:]
        vmax1[i]         = V_tuple1[i,np.where(vmax[i] == V_tuple[i,:])[0][0]]




In [1477]:
vmax_mat = vmax.reshape(N_grid_a, N_grid_a)
vmax0    = np.nanmax(vmax_mat,1)

pol_a_mat = pol_func_a[:,1].reshape(N_grid_a, N_grid_a)
pol_b_mat = pol_func_b[:,2].reshape(N_grid_a, N_grid_a)
pol_c_mat = pol_func_c[:,1].reshape(N_grid_a, N_grid_a)




pol_a     = vmax0.copy()
pol_b     = vmax0.copy()
pol_c     = vmax0.copy()

In [1478]:
for i in range(N_grid_a):
    pol_a[i]     = np.nan
    pol_a[i]     = pol_a_mat[i,:][np.where(vmax0[i] == vmax_mat[i,:])[0][0]]
    pol_b[i]     = np.nan
    pol_b[i]     = pol_b_mat[i,:][np.where(vmax0[i] == vmax_mat[i,:])[0][0]]
    pol_c[i]     = np.nan
    pol_c[i]     = pol_c_mat[i,:][np.where(vmax0[i] == vmax_mat[i,:])[0][0]]
